In [1]:
from IPython.core.display import HTML
HTML('''
<script>
var logoParent = document.getElementById("kernel_logo_widget")
var logo = document.getElementById("kernel_logo_widget").getElementsByClassName("current_kernel_logo")[0];
logo.src = "https://i.ibb.co/mD4jTGQ/itclogo.jpg";
logo.style = "display: inline; width:138px; height:40px";
logoParent.innerHTML = '<a href="https://i.ibb.co/mD4jTGQ/itclogo.jpg">' + logoParent.innerHTML + '</a>';
</script>
''')

<font size="36"><b>Pandas - Multi-Index and Aggregation Assignment</b></font> <img src = "https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/03/pandas.jpg" height=200 width=200>

In [2]:
import pandas as pd

For this assignement, we will work with the [tips dataset](https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv). 

In [3]:
df_tips = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv')

# Assignment

Answer the following questions:

 1. Calculate the average % of tips out of the total bill that Females leave, vs. Men. Present the data in one dataframe
 2. Calculate the average % of tips out of the total bill that Smoker Females leave, vs. Smoker Men, vs. non-smoking Men and Females. Present the data in one dataframe
 3. Find the day and the time in which the most tip is paid (in absolute value)
 4. Create an aggregative function that finds the top 5 highest vaslues in a series, and returns their sum's % out of the entire series. Use in on the data with the `agg` operator, to find the top 5 bills average, out of the total bills in each time and day
 5. Calculate the sum of bills per group: non-smoking Men, non-smoking Females, smoking Females and smoking Men. Code it so you get a data frame with the columns "Female" and "Male", below a larger column "total_bill" (hierarchical index)
 6. Add another index "day" to Q5, do that we can see the conclusion per day per smokers group per gender
 7. Take the result from Q6, and now aggregate by `mean`, `std`, and `sum`
 8. Create a series of plots that present your findings from Q7. Make your plots informative. Make sure that all of the conclusions from the dataframe can be seen in the series of graphs.

<div class="alert alert-info"><b>Note:</b> Try to answer the questions with one line of code, using the '.' pipe operator, when possible</div>

## Q1

In [6]:
def tip_per(x):
    return x.tip/x.total_bill*100

pd.DataFrame(df_tips.groupby('sex').apply(tip_per).mean(level=0), columns = ['average tip%']).reset_index()

,sex,average tip%
0,Female,16.649074
1,Male,15.765055


## Q2

In [7]:
pd.DataFrame(df_tips.groupby(['sex','smoker']).apply(tip_per).mean(level=[0,1]),columns = ['average tip%']).reset_index()

,sex,smoker,average tip%
0,Female,No,15.692097
1,Female,Yes,18.215035
2,Male,No,16.066872
3,Male,Yes,15.277118


## Q3

In [8]:
df_tips.iloc[df_tips['tip'].idxmax()][['day','time']]

day        Sat
time    Dinner
Name: 170, dtype: object

## Q4

In [9]:
#This question is doable in 1 line by piping but I chose to do it on 2 lines to make it more readable.
def aggr_func(my_series):
    return my_series.nlargest(5).sum()/my_series.sum()*100 

top5_bill_df = pd.DataFrame(df_tips.groupby(['time','day'])['total_bill'].agg(aggr_func)).reset_index()
top5_bill_df.rename(columns = {'total_bill': '% average top 5 bills'},inplace = True)
top5_bill_df

,time,day,% average top 5 bills
0,Dinner,Fri,60.035599
1,Dinner,Sat,12.996514
2,Dinner,Sun,12.746134
3,Dinner,Thur,100.000000
4,Lunch,Fri,79.237100
5,Lunch,Thur,17.271588


## Q5

In [10]:
#This question is doable on 1 line by piping but I chose to do it on 2 lines to make it more readable.
df_tips_sum = df_tips.groupby(['smoker','sex']).aggregate(sum).total_bill
df_unstacked = pd.DataFrame(df_tips_sum).unstack()
df_unstacked

total_bill         
sex        Female     Male
smoker                    
No         977.68  1919.75
Yes        593.27  1337.07

## Q6

In [15]:
#This question is doable on 1 line by piping but I chose to do it on 2 lines to make it more readable.
df_tips_sum = df_tips.groupby(['smoker', 'day','sex']).aggregate(sum).total_bill
df_unstacked = pd.DataFrame(df_tips_sum).unstack()
df_unstacked

total_bill        
sex             Female    Male
smoker day                    
No     Fri       38.73   34.95
       Sat      247.05  637.73
       Sun      291.54  877.34
       Thur     400.36  369.73
Yes    Fri       88.58  163.62
       Sat      304.00  589.62
       Sun       66.16  392.12
       Thur     134.53  191.71

# Q7

In [36]:
#This question is doable on 1 line by piping but I chose to do it on 2 lines to make it more readable.
df_tips_sum = df_tips.groupby(['smoker', 'day','sex']).aggregate(['mean','std','sum']).total_bill
df_unstacked = pd.DataFrame(df_tips_sum).unstack()
df_unstacked

mean                   std                sum        
sex             Female       Male     Female       Male  Female    Male
smoker day                                                             
No     Fri   19.365000  17.475000   4.787113   7.092281   38.73   34.95
       Sat   19.003846  19.929063   6.730219   9.779061  247.05  637.73
       Sun   20.824286  20.403256   8.396159   8.140559  291.54  877.34
       Thur  16.014400  18.486500   6.783939   8.739134  400.36  369.73
Yes    Fri   12.654286  20.452500   3.883138  10.943815   88.58  163.62
       Sat   20.266667  21.837778  10.485703   9.988045  304.00  589.62
       Sun   16.540000  26.141333   4.854764  10.693824   66.16  392.12
       Thur  19.218571  19.171000  10.847137   6.757421  134.53  191.71

# Q8

In [40]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

data = df_tips.groupby(['smoker', 'day','sex'])
data
# fig7, ax7 = plt.subplots()
# ax7.set_title('Multiple Samples with Different sizes')
# ax7.boxplot(data)

# plt.show()
# df_tips_sum = df_tips.groupby(['smoker', 'day','sex'])['total_bill'].get_group(
# print(df_tips_sum)
# df_unstacked = pd.DataFrame(df_tips_sum).unstack()
# df_unstacked
fig = plt.figure(1, figsize=(20, 10))
ax = fig.add_subplot(221)
df_tips[df_tips['day']=='Thur'].boxplot(column=['total_bill'], by=['sex', 'smoker'], ax=ax, showmeans=True)
ax.set(title="Total bills on Thursday by sex and smoker", xlabel = "", ylabel = "total_bill")
ax = fig.add_subplot(222)
df_tips[df_tips['day']=='Fri'].boxplot(column=['total_bill'], by=['sex', 'smoker'], ax=ax, showmeans=True)
ax.set(title="Total bills on Friday by sex and smoker", xlabel = "", ylabel = "total_bill")
ax = fig.add_subplot(223)
df_tips[df_tips['day']=='Sat'].boxplot(column=['total_bill'], by=['sex', 'smoker'], ax=ax, showmeans=True)
ax.set(title="Total bills on Saturday by sex and smoker", xlabel = "", ylabel = "total_bill")
ax = fig.add_subplot(224)
df_tips[df_tips['day']=='Sun'].boxplot(column=['total_bill'], by=['sex', 'smoker'], ax=ax, showmeans=True)
ax.set(title="Total bills on Sunday by sex and smoker", xlabel = "", ylabel = "total_bill")
fig.tight_layout()
fig.subplots_adjust(top=0.85)
plt.suptitle("Total bill for each day", size=16)